In [42]:
import pandas as pd

In [43]:
data = pd.read_csv('../input/train/train.csv')

In [44]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [45]:
data.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [46]:
data = data.dropna()

In [47]:
x = data.drop('label', axis=1)

In [48]:
y = data['label']

In [49]:
x.shape

(18285, 4)

In [50]:
y.shape

(18285,)

In [51]:
import tensorflow as tf

In [52]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [53]:
voc_size = 5000

# onebot representation

In [54]:
messages = x.copy()

In [55]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [56]:
messages.reset_index(inplace=True)

In [57]:
import nltk
import re
from nltk.corpus import stopwords

In [58]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [59]:
#dataset preprocessing

In [60]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
        
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [61]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [62]:
onehot_repr = [one_hot(words, voc_size)for words in corpus]
onehot_repr

[[4453, 2618, 3276, 2551, 1345, 1220, 517, 3015, 1876, 31],
 [1070, 2883, 990, 667, 577, 1796, 2204],
 [3252, 3092, 645, 3091],
 [524, 3830, 1917, 904, 2797, 834],
 [4474, 577, 2461, 1439, 2874, 2952, 577, 2487, 1167, 750],
 [1307,
  2463,
  550,
  3822,
  2325,
  4420,
  4589,
  3778,
  1921,
  2598,
  3044,
  3299,
  388,
  2613,
  2204],
 [178, 1830, 4612, 4582, 524, 4528, 408, 834, 4238, 2421, 2519],
 [2557, 2829, 2133, 1351, 2077, 479, 4420, 1521, 4238, 2421, 2519],
 [2614, 2576, 3110, 3121, 2261, 601, 902, 4384, 4420, 1964],
 [4724, 3489, 4966, 2656, 4383, 3325, 2717, 4341],
 [4282, 3407, 3230, 3959, 4387, 4891, 1020, 2672, 2087, 671, 1243],
 [904, 3704, 1345, 601, 4420, 2077],
 [1540, 3439, 4286, 4280, 529, 1962, 2938, 2049, 3752],
 [4427, 4866, 2845, 345, 2547, 4292, 3041, 4238, 2421, 2519],
 [2448, 4625, 3151, 4629, 1629, 4238, 2421, 2519],
 [903, 1238, 2534, 1129, 4878, 4175, 4648, 1706, 2104, 4926],
 [2761, 2745, 2883],
 [1352, 1286, 4976, 517, 4420, 669, 718, 2204],
 [2338,

# Embedding Representation

In [63]:
sent_length = 20
embedded_docs =pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 3015, 1876,   31],
       [   0,    0,    0, ...,  577, 1796, 2204],
       [   0,    0,    0, ..., 3092,  645, 3091],
       ...,
       [   0,    0,    0, ..., 4238, 2421, 2519],
       [   0,    0,    0, ..., 1366,  460, 2660],
       [   0,    0,    0, ...,  154, 1786,  542]], dtype=int32)

In [64]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4453,
       2618, 3276, 2551, 1345, 1220,  517, 3015, 1876,   31], dtype=int32)

# Creating model

In [65]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [66]:
len(embedded_docs),y.shape

(18285, (18285,))

In [67]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [68]:
x_final.shape, y_final.shape

((18285, 20), (18285,))

In [69]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [70]:
model.fit(x_train,y_train,validation_data=(x_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 27ms/step - loss: 0.3286 - accuracy: 0.8400 - val_loss: 0.1928 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1381 - accuracy: 0.9468 - val_loss: 0.1925 - val_accuracy: 0.9195
Epoch 3/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0929 - accuracy: 0.9654 - val_loss: 0.2450 - val_accuracy: 0.9231
Epoch 4/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0599 - accuracy: 0.9802 - val_loss: 0.2839 - val_accuracy: 0.9150
Epoch 5/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0402 - accuracy: 0.9866 - val_loss: 0.3033 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0224 - accuracy: 0.9934 - val_loss: 0.3330 - val_accuracy: 0.9127
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0140 - accuracy: 0.9958 - val_loss: 0.4394 - val_accuracy: 0.9143

# Adding Dropout

In [71]:
from tensorflow.keras.layers import Dropout

embedding_vector_features=40
model_1=Sequential()
model_1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model_1.add(Dropout(0.3))
model_1.add(LSTM(100))
model_1.add(Dropout(0.3))
model_1.add(Dense(1,activation='sigmoid'))
model_1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [72]:
model_1.fit(x_train,y_train,validation_data=(x_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 29ms/step - loss: 0.3343 - accuracy: 0.8441 - val_loss: 0.1991 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1520 - accuracy: 0.9399 - val_loss: 0.1895 - val_accuracy: 0.9216
Epoch 3/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1114 - accuracy: 0.9591 - val_loss: 0.2013 - val_accuracy: 0.9200
Epoch 4/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0810 - accuracy: 0.9708 - val_loss: 0.2568 - val_accuracy: 0.9165
Epoch 5/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0620 - accuracy: 0.9786 - val_loss: 0.2619 - val_accuracy: 0.9171
Epoch 6/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0471 - accuracy: 0.9833 - val_loss: 0.2824 - val_accuracy: 0.9148
Epoch 7/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0325 - accuracy: 0.9894 - val_loss: 0.3541 - val_accuracy: 0.9132

# Performance metrics and accuracy

In [73]:
y_pred=model_1.predict_classes(x_test)

In [74]:
from sklearn.metrics import confusion_matrix

In [75]:
confusion_matrix(y_test,y_pred)

array([[3090,  329],
       [ 180, 2436]])

In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9156586578293289